将句中识别的实体与知识库中实体进行匹配，解决实体歧义问题。 可利用上下文本相似度进行识别。

在data/entity_disambiguation目录中，entity_list.csv是50个实体，valid_data.csv是需要消歧的语句。

答案提交在submit目录中，命名为entity_disambiguation_submit.csv。格式为：第一列是需要消歧的语句序号，第二列为多个“实体起始位坐标-实体结束位坐标：实体序号”以“|”分隔的字符串。

In [2]:
import jieba
import pandas as pd

# TODO：将entity_list.csv中已知实体的名称导入分词词典
entity_data = pd.read_csv('../data/entity_disambiguation/entity_list.csv', encoding = 'utf-8')


# TODO：对每句句子识别并匹配实体     
valid_data = pd.read_csv('../data/entity_disambiguation/valid_data.csv', encoding = 'gb18030')
    

In [3]:
# 建立关键词组
import collections

s = ''
keyword_list = []
for i in entity_data['entity_name'].values.tolist():
    s += i + '|'
for k,v in collections.Counter(s.split('|')).items():
    if v > 1:
        keyword_list.append(k)

In [4]:
# 生成tfidf矩阵
from sklearn.feature_extraction.text import TfidfVectorizer

train_sentence = []
for i in entity_data['desc'].values:
    train_sentence.append(' '.join(jieba.cut(i)))

vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(train_sentence)

Building prefix dict from the default dictionary ...
Loading model from cache /var/folders/1y/y3wkm_bd157fxlr0npb5zv9r0000gn/T/jieba.cache
Loading model cost 1.146 seconds.
Prefix dict has been built succesfully.


In [5]:
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

# 获取包含关键词的句子中关键词所属的entity_id
def get_entityid(sentence):
    id_start = 1001 
    a_list = [' '.join(jieba.cut(sentence))]
    res = cosine_similarity(vectorizer.transform(a_list),X)[0]
    top_idx = np.argsort(res)[-1]
    return id_start + top_idx

In [6]:
# TODO：将计算结果存入文件

row = 0
result_data = []
neighbor_sentence = ''
for sentence in valid_data['sentence']:
    res = [row]
    for keyword in keyword_list:
        if keyword in sentence:
            # 查询关键词在句子中的索引
            k_len = len(keyword)
            ss =''
            for i in range(len(sentence)-k_len+1):
                if sentence[i:i+k_len] == keyword:
                    s = str(i) + '-' +str(i+k_len) + ':'  # 拿到 x-x:
                    if i > 10 and i + k_len < len(sentence)-9:
                        neighbor_sentence = sentence [i-10:i+k_len+9]
                    elif i < 10:
                        neighbor_sentence = sentence [:20]
                    elif i + k_len > len(sentence)-9:
                        neighbor_sentence = sentence [-20:]
                    s +=  str(get_entityid(neighbor_sentence))   # 拿到 x-x:id
                    ss += s + '|'    # 拿到 x-x:id|x-x:id
            res.append(ss[:-1])  # 拼接成[0, 'x-x:id|x-x:id']
    result_data.append(res)
    row += 1
pd.DataFrame(result_data).to_csv('../submit/entity_disambiguation_submit.csv', index=False)


In [7]:
pd.read_csv('../submit/entity_disambiguation_submit.csv')

,0,1
0,0,3-6:1008|109-112:1008|187-190:1008
1,1,18-21:1008
2,2,23-26:1006|40-43:1006
3,3,7-10:1006
4,4,2-5:1006|14-17:1006
5,5,28-30:1001|34-36:1001|41-43:1001
6,6,4-6:1001|25-27:1001|34-36:1001|94-96:1001|100-...
7,7,0-2:1001|6-8:1001|19-21:1009|34-36:1001|45-47:...
8,8,8-10:1001|22-24:1001|34-36:1001|37-39:1001|46-...
9,9,14-16:1002
